In [523]:
import pandas as pd
import os
import numpy as np
import re

Actigraph

In [524]:
outputDir = r'R:/KumarLab3/PROJECTS/wesens/Data/Analysis/Kim_E/Pain Trajectory/Data/'

In [525]:
# Remove W
def extract_number(num):
    match = re.search(r'\d+', num)
    if match:
        return int(match.group())
    return None

In [526]:
lcmm_group = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/Data/Analysis/Kim_E/Pain Trajectory/Results/ClassMembership_total.csv")
profile = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/Data/Analysis/Kim Fx MEP/Statistical An/construct/DataSet/mep_func_stg_qst_merged_11-07-2024.csv")
walk6m_gait = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/Data/Analysis/Kim_E/apdm_6mwt_28m_wesens_fnirs_tempo/DataSet/mep_gait_minute_6mwt.csv")

In [527]:
demo_data = profile[profile['project']== 'wesens']
demo_data['record_id'] = demo_data['record_id'].apply(extract_number)

filtered_profile = demo_data[demo_data['record_id'].isin(lcmm_group['record_id'])]
combined_data = pd.merge(lcmm_group, filtered_profile, on='record_id', how='outer')

C:\Users\kimawc\AppData\Local\Temp\ipykernel_26796\108187174.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  demo_data['record_id'] = demo_data['record_id'].apply(extract_number)


In [528]:
walk6m_gait = walk6m_gait[walk6m_gait['minute']=="all"]
gait = walk6m_gait[walk6m_gait['record_id'].str.startswith('W', na=False)].copy()
gait['record_id'] = gait['record_id'].apply(extract_number)
gait['week'] = 'baseline'
filtered_profile2 = gait[gait['record_id'].isin(lcmm_group['record_id'])]
combined_data = pd.merge(combined_data, filtered_profile2, on=['record_id', 'week'], how='outer')

In [529]:
sleep_visit = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/BEDAC/external/H_WESENS_GT9X_Sleep/gt9x_sleep_visit_valid_an.csv")
sleep_visit['visit'] = sleep_visit['visit'].replace({
    'BL (baseline)': 'baseline', 
    'W06 (week 6)': 'week6', 
    'W12 (week 12)': 'week12', 
    'W18 (week 18)': 'week18'
})
sleep_visit.rename(columns={'visit': 'week'}, inplace=True)
sleep_visit.replace('.', np.nan, inplace=True)

In [530]:
filtered_profile3 = sleep_visit[sleep_visit['record_id'].isin(lcmm_group['record_id'])]
combined_data = pd.merge(combined_data, filtered_profile3, on=['record_id', 'week'], how='outer')

In [531]:
columns_to_drop = ['6mwt_nrs', '6mwt_mep', '6mwt_mep_minute']
combined_data = combined_data.drop(columns=columns_to_drop)
combined_data.to_csv(os.path.join(outputDir,"sleep_profile_gait_eff_tst_waso.csv"), index=False)

## GT9X

In [532]:
# Actigraph - daily
sleep_daily_df = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/BEDAC/external/H_WESENS_GT9X_Sleep/gt9x_sleep_daily_valid_an.csv")

sleep_col = ['record_id', 'visit','day_from_first_ptvisit', 'date',	'calendar_week',
           'Efficiency', 'tib','tst', 'waso',	'Number_of_Awakenings', 'Sleep_Fragmentation_Index', 'MovementIndex', 'FragmentationIndex']

sleep_daily_df['visit'] = sleep_daily_df['visit'].replace({
    'BL (baseline)': 'baseline', 
    'W06 (week 6)': 'week6', 
    'W12 (week 12)': 'week12', 
    'W18 (week 18)': 'week18'
})

sleep_daily_df = sleep_daily_df[sleep_col]
sleep_daily_df = sleep_daily_df.replace('.', np.nan)

sleep_daily_df = sleep_daily_df.sort_values(by=['record_id', 'day_from_first_ptvisit'])
sleep_daily_df = sleep_daily_df.reset_index(drop=True)

# Find the calendar week 
sleep_daily_df['calendar_week_from_first_pt_vist'] = pd.NA
days = [1, 7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 91, 98, 105, 112, 119, 126, 133, 140, 147, 152]
for day in days:
    day_0 = day - 1
    sleep_daily_df.loc[sleep_daily_df['day_from_first_ptvisit'] == day_0, 'calendar_week_from_first_pt_vist'] = sleep_daily_df.loc[sleep_daily_df['day_from_first_ptvisit'] == day_0, 'calendar_week']

visit = ['baseline', 'week6', 'week12', 'week18']
for day in visit:
    sleep_daily_df.loc[sleep_daily_df['visit'] == day, 'calendar_week_from_first_pt_vist'] = sleep_daily_df['calendar_week']
sleep_daily_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_daily.csv'), index=False)

In [533]:
# sleep_daily_lcmm_df = pd.merge(lcmm_group, sleep_daily_df, on=['record_id'], how = 'outer')
# sleep_daily_lcmm_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_lcmm__beta4_daily.csv'), index=False)

In [534]:
sleep_daily_df2 = sleep_daily_df[['record_id', 'day_from_first_ptvisit', 'calendar_week']]
sleep_daily_df2['day_from_first_ptvisit'] = pd.to_numeric(sleep_daily_df2['day_from_first_ptvisit'], errors='coerce')
sleep_daily_df2 = sleep_daily_df2[sleep_daily_df2['day_from_first_ptvisit'] >= 0]
sleep_daily_df2 = sleep_daily_df2.dropna(subset=['calendar_week'])
sleep_daily_df2 = sleep_daily_df2.drop_duplicates(subset=['record_id', 'calendar_week'])
sleep_daily_df2 = sleep_daily_df2.reset_index(drop=True)

C:\Users\kimawc\AppData\Local\Temp\ipykernel_26796\1986421849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep_daily_df2['day_from_first_ptvisit'] = pd.to_numeric(sleep_daily_df2['day_from_first_ptvisit'], errors='coerce')


In [541]:
sleep_baseline = sleep_daily_df[['record_id', 'visit', 'day_from_first_ptvisit', 'calendar_week']]
sleep_baseline = sleep_baseline[sleep_baseline['visit']=='baseline']
sleep_baseline['redcap_event_name'] = 0
sleep_baseline = sleep_baseline.drop_duplicates(subset=['record_id', 'calendar_week'])
sleep_baseline = sleep_baseline.reset_index(drop=True)

,record_id,visit,day_from_first_ptvisit,calendar_week,redcap_event_name
0,2,baseline,-18,12,0
1,2,baseline,-15,13,0
2,3,baseline,-32,3,0
3,3,baseline,-30,4,0
4,10,baseline,-14,28,0
...,...,...,...,...,...
86,115,baseline,-22,21,0
87,117,baseline,-33,19,0
88,117,baseline,-29,20,0
89,127,baseline,-18,20,0


In [ ]:
def assign_event_name_with_gap(group):
    redcap_event_name = []
    current_event = 1  # Start from 1
    
    # Track the previous calendar week to detect gaps
    prev_week = None
    
    for week in group['calendar_week']:
        if prev_week is not None:
            # Check if there's a gap larger than 1 week
            if week > prev_week + 1:
                current_event += (week - prev_week - 1)
        redcap_event_name.append(current_event)
        current_event += 1  # Increment for next week
        prev_week = week  # Update previous week
    
    group['redcap_event_name'] = redcap_event_name
    return group

sleep_daily_df2 = sleep_daily_df2.groupby('record_id', group_keys=False).apply(assign_event_name_with_gap)

In [ ]:
sleep_daily_df2 = pd.concat([sleep_baseline, sleep_daily_df2])
sleep_daily_df2 = sleep_daily_df2.sort_values(by=['record_id', 'redcap_event_name']).reset_index(drop=True)

In [ ]:
# Actigraph - weekly
sleep_weekly_df = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/BEDAC/external/H_WESENS_GT9X_Sleep/gt9x_sleep_weekly_valid_an.csv")
sleep_weekly_df = sleep_weekly_df.sort_values(by=['record_id', 'calendar_week'])
sleep_weekly_df = pd.merge(sleep_daily_df2, sleep_weekly_df, on = ['record_id', 'calendar_week'], how = 'outer')

In [ ]:
sleep_col = ['record_id', 'calendar_week', 'redcap_event_name', 
             'Efficiency_Mean', 'Efficiency_Median','Efficiency_Mode', 'Efficiency_StdDev', 'Efficiency_Min','Efficiency_Max', 'Efficiency_N', 'Efficiency_NMiss', 
             'tst_Mean', 'tst_Median', 'tst_Mode', 'tst_StdDev','tst_Min', 'tst_Max', 'tst_N', 'tst_NMiss',
             'waso_Mean', 'waso_Median','waso_Mode', 'waso_StdDev', 'waso_Min', 'waso_Max', 'waso_N','waso_NMiss', 
             'Number_of_Awakenings_Mean','Number_of_Awakenings_Median', 'Number_of_Awakenings_Mode','Number_of_Awakenings_StdDev', 'Number_of_Awakenings_Min','Number_of_Awakenings_Max', 'Number_of_Awakenings_N','Number_of_Awakenings_NMiss',
             'Sleep_Fragmentation_Index_Mean','Sleep_Fragmentation_Ind_Median', 'Sleep_Fragmentation_Index_Mode','Sleep_Fragmentation_Ind_StdDev', 'Sleep_Fragmentation_Index_Min','Sleep_Fragmentation_Index_Max', 'Sleep_Fragmentation_Index_N',
             'Sleep_Fragmentation_Inde_NMiss']


sleep_visit_weekly_df = sleep_weekly_df[sleep_col]
sleep_visit_weekly_df = sleep_visit_weekly_df.replace('.', np.nan)
sleep_visit_weekly_df['redcap_event_name'] = pd.to_numeric(sleep_visit_weekly_df['redcap_event_name'], errors='coerce')

sleep_visit_weekly_df = sleep_visit_weekly_df.sort_values(by=['record_id', 'redcap_event_name'])
sleep_visit_weekly_df = sleep_visit_weekly_df.reset_index(drop=True)
sleep_visit_weekly_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_weekly.csv'), index=False)

## KOOS + DSIS

In [ ]:
# Intervention Visit Day 
df_epro1 = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/BEDAC/redcap/wesensoa_main_blw6w12w18et_an.csv")

df_visitday = df_epro1[['record_id', 'redcap_event_name', 'tx_hx_visitdate_day']]
df_visitday = df_visitday[(df_visitday['redcap_event_name'] != 'At-Home Baseline Visit') & (df_visitday['redcap_event_name'] != 'Screening and Enrollment') & (df_visitday['redcap_event_name'] != 'Early Termination')]
df_visitday['redcap_event_name'] = df_visitday['redcap_event_name'].replace({
    'Baseline Visit': 'Baseline Visit',
    'Week 6': '6', 'Week 12': '12', 'Week 18': '18'
})

# DSIS
df_dsis = pd.read_csv(r"R:/KumarLab3/PROJECTS/wesens/BEDAC/redcap/wesensoa_main_dailyweekly_an.csv")
dsis_df = df_dsis.copy()

dsis_daily_col = ['record_id','redcap_repeat_instance_dv', 'redcap_repeat_instrument', 'dsis_02bdbc']
dsis_weely_col = ['record_id','redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance', 'dsis_02bdbc']
koos_weely_col = ['record_id','redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance_dv',
                  'koos_pain',	'koos_adl',	'kd_womac_pain',	'kd_womac_stiff',	'kd_womac_func' ]
pgaoa_weely_col = ['record_id','redcap_event_name', 'redcap_repeat_instrument', 'redcap_repeat_instance_dv','pgaoa']

# Replace 
dsis_df.loc[dsis_df['redcap_event_name'] == 'Baseline Visit', 'redcap_repeat_instance_dv'] = dsis_df['redcap_repeat_instance_dv'].replace('.', 0)
dsis_df.loc[(dsis_df['redcap_event_name'] == 'Baseline Visit') & (dsis_df['dsis_02bdbc'] == '.'), 'dsis_02bdbc'] = dsis_df['dsis2']

In [ ]:
# PGAOA Weekly
pgaoa_weekly_df = dsis_df[pgaoa_weely_col]
pgaoa_weekly_df.loc[pgaoa_weekly_df['redcap_event_name'] == 'Baseline Visit', 'redcap_repeat_instance_dv'] = pgaoa_weekly_df['redcap_repeat_instance_dv'].replace('.', 'Baseline Visit')

pgaoa_weekly_df = pgaoa_weekly_df[pgaoa_weekly_df['redcap_event_name'].isin(['Weekly','Baseline Visit'])]
pgaoa_weekly_df = pgaoa_weekly_df[pgaoa_weekly_df['redcap_repeat_instrument'].isin(['PGAOA', np.nan])]
pgaoa_weekly_df = pgaoa_weekly_df.drop(columns=['redcap_event_name', 'redcap_repeat_instrument'])

# KOOS Weekly
koos_weekly_df = dsis_df[koos_weely_col]
koos_weekly_df.loc[koos_weekly_df['redcap_event_name'] == 'Baseline Visit', 'redcap_repeat_instance_dv'] = koos_weekly_df['redcap_repeat_instance_dv'].replace('.', 'Baseline Visit')

koos_weekly_df = koos_weekly_df[koos_weekly_df['redcap_event_name'].isin(['Weekly','Baseline Visit'])]
koos_weekly_df = koos_weekly_df[koos_weekly_df['redcap_repeat_instrument'].isin(['Koosweekly', np.nan])]
koos_weekly_df = koos_weekly_df.drop(columns=['redcap_event_name', 'redcap_repeat_instrument'])

# MERGE
koos_pgaoa_weekly_df = pd.merge(koos_weekly_df, pgaoa_weekly_df, on = ['record_id', 'redcap_repeat_instance_dv'],  how = 'outer')
koos_pgaoa_weekly_df['pgaoa'] = koos_pgaoa_weekly_df['pgaoa'].replace('.', np.nan)
koos_pgaoa_weekly_df.replace('.', np.nan)

koos_pgaoa_weekly_df = koos_pgaoa_weekly_df. rename(columns = {'redcap_repeat_instance_dv': 'redcap_event_name'})
# koos_weekly_df = pd.merge(koos_weekly_df, df_visitday, on = ['record_id', 'redcap_event_name'],  how = 'inner')
# koos_weekly_df['redcap_event_name'] = koos_weekly_df['redcap_event_name'].replace('Baseline Visit', 0)

# # as.numeric
# koos_weekly_df['tx_hx_visitdate_day'] = pd.to_numeric(koos_weekly_df['tx_hx_visitdate_day'], errors='coerce')
koos_pgaoa_weekly_df['redcap_event_name'] = pd.to_numeric(koos_pgaoa_weekly_df['redcap_event_name'], errors='coerce')
koos_pgaoa_weekly_df.to_csv(os.path.join(outputDir, 'koos_pgaoa_weekly.csv'), index=False)

DSIS

In [ ]:
# DSIS daily
dsis_daily_df = dsis_df[dsis_daily_col]
dsis_daily_df = dsis_daily_df[dsis_daily_df['redcap_repeat_instrument'].isin(['Daily Log', 'Weekly', np.nan])]
dsis_daily_df = dsis_daily_df.rename(columns = {'redcap_repeat_instance_dv': 'redcap_event_name', 'dsis_02bdbc': 'dsis'})
dsis_daily_df['redcap_event_name'] = pd.to_numeric(dsis_daily_df['redcap_event_name'], errors='coerce')
dsis_daily_df = dsis_daily_df.sort_values(by=['record_id', 'redcap_event_name'])
dsis_daily_df = dsis_daily_df. drop(columns = {'redcap_repeat_instrument'})

# as.numeric
dsis_daily_df['dsis'] = dsis_daily_df['dsis'].replace('.', np.nan)
dsis_daily_df['dsis'] = dsis_daily_df['dsis'].replace('0 Did not interfere with sleep', '0')
# dsis_daily_df.to_csv(os.path.join(outputDir, 'dsis_daily.csv'), index=False)

In [ ]:
# Baseline
dsis_baseline = dsis_daily_df[dsis_daily_df['redcap_event_name']==0]
dsis_baseline = dsis_baseline.reset_index(drop=True)

In [ ]:
# DSIS Weekly
dsis_weekly_df = dsis_df[dsis_weely_col]

dsis_weekly_df = dsis_weekly_df[dsis_weekly_df['redcap_event_name'].isin(['Weekly'])]
dsis_weekly_df = dsis_weekly_df[dsis_weekly_df['redcap_repeat_instrument'].isin(['Daily Log', np.nan])]
dsis_weekly_df = dsis_weekly_df.drop(columns=['redcap_event_name', 'redcap_repeat_instrument'])
dsis_weekly_df = dsis_weekly_df. rename(columns = {'dsis_02bdbc': 'dsis'})
dsis_weekly_df = dsis_weekly_df. rename(columns = {'redcap_repeat_instance': 'redcap_event_name'})
dsis_weekly_df = dsis_weekly_df.reset_index(drop=True)
# dsis_weekly_df = pd.merge(dsis_weekly_df, df_visitday, how = 'outer', on = ['record_id', 'redcap_event_name'])
# dsis_weekly_df['redcap_repeat_instance'] = dsis_weekly_df['redcap_repeat_instance'].replace('.', 0)

# as.numeric
# dsis_weekly_df['redcap_event_name'] = pd.to_numeric(dsis_weekly_df['redcap_event_name'], errors='coerce')
# dsis_weekly_df['tx_hx_visitdate_day'] = pd.to_numeric(dsis_weekly_df['tx_hx_visitdate_day'], errors='coerce')
dsis_weekly_df['redcap_event_name'] = pd.to_numeric(dsis_weekly_df['redcap_event_name'], errors='coerce')

dsis_weekly_df['dsis'] = dsis_weekly_df['dsis'].replace('.', np.nan)
dsis_weekly_df['dsis'] = dsis_weekly_df['dsis'].replace('0 Did not interfere with sleep', '0')
dsis_weekly_df = pd.concat([dsis_baseline, dsis_weekly_df])
dsis_weekly_df = dsis_weekly_df.sort_values(by=['record_id', 'redcap_event_name'])
dsis_weekly_df.to_csv(os.path.join(outputDir, 'dsis_redcap_weekly.csv'), index=False)

In [ ]:
# CALCULATE WEEKLY BASED ON ... 
dsis_weekly_cal_df = dsis_daily_df.copy()
dsis_weekly_cal_df = dsis_weekly_cal_df[dsis_weekly_cal_df["redcap_event_name"] != 0]
dsis_weekly_cal_df["week"] = (dsis_weekly_cal_df["redcap_event_name"]//7) + 1
dsis_weekly_cal_df['dsis'] = pd.to_numeric(dsis_weekly_cal_df['dsis'], errors='coerce')

weekly_stats = dsis_weekly_cal_df.groupby(["record_id", "week"]).agg(
    dsis_mean=("dsis", "mean"),
    dsis_sd=("dsis", "std"),
    dsis_median=("dsis", "median"),
    dsis_missing_n=("dsis", lambda x: x.isna().sum())
).reset_index()

weekly_stats.loc[weekly_stats["dsis_missing_n"] > 3, ["dsis_mean", "dsis_sd", "dsis_median"]] = None
weekly_stats.loc[weekly_stats["dsis_missing_n"] == 7, ["dsis_mean", "dsis_sd", "dsis_median"]] = None
weekly_stats = weekly_stats.rename(columns = {'week': 'redcap_event_name'})
# weekly_stats['redcap_event_name'] = weekly_stats['redcap_event_name'].astype(str)
weekly_stats = weekly_stats.sort_values(by=['record_id', 'redcap_event_name'])
weekly_stats.to_csv(os.path.join(outputDir, 'dsis_mean_weekly.csv'), index=False)

MERGE weekly daily

In [ ]:
# WEEKLY
gt9x_dsis_weekly_df = pd.merge(dsis_weekly_df, weekly_stats, how= 'outer', on=['record_id', 'redcap_event_name'])
gt9x_dsis_weekly_df = pd.merge(gt9x_dsis_weekly_df, sleep_visit_weekly_df, how= 'outer', on=['record_id', 'redcap_event_name'])
gt9x_dsis_weekly_df = gt9x_dsis_weekly_df.sort_values(by=['record_id', 'redcap_event_name'])
gt9x_dsis_weekly_df = gt9x_dsis_weekly_df.reset_index(drop=True)
gt9x_dsis_weekly_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_dsis_weekly.csv'), index=False)

In [ ]:
# WEEKLY with KOOS
gt9x_dsis_weekly_df = pd.merge(gt9x_dsis_weekly_df, koos_pgaoa_weekly_df, how= 'outer', on=['record_id', 'redcap_event_name'])
gt9x_dsis_weekly_df = gt9x_dsis_weekly_df.sort_values(by=['record_id', 'redcap_event_name'])
gt9x_dsis_weekly_df = gt9x_dsis_weekly_df.reset_index(drop=True)
gt9x_dsis_weekly_df['redcap_event_name'] = pd.to_numeric(gt9x_dsis_weekly_df['redcap_event_name'], errors='coerce')
gt9x_dsis_weekly_df = gt9x_dsis_weekly_df.sort_values(by=['record_id',  'redcap_event_name']).reset_index(drop=True)
gt9x_dsis_weekly_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_dsis_koos_pgaoa_weekly.csv'), index=False)

In [ ]:
# DAILY
sleep_daily_df['redcap_event_name'] = sleep_daily_df['day_from_first_ptvisit']

gt9x_dsis_daily_df = pd.merge(dsis_daily_df, sleep_daily_df, how= 'outer', on=['record_id', 'redcap_event_name'])
gt9x_dsis_daily_df = gt9x_dsis_daily_df.sort_values(by=['record_id', 'redcap_event_name'])
gt9x_dsis_daily_df = gt9x_dsis_daily_df.reset_index(drop=True)
gt9x_dsis_daily_df.to_csv(os.path.join(outputDir, 'gt9x_sleep_dsis_daily.csv'), index=False)